In [1]:
import pandas as pd
import numpy as np

from models import XGBClassifier, RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split

from metrics import *
from sklearn.metrics import f1_score

path_to_csv = "creditor_rating.csv"
df = pd.read_csv(path_to_csv, sep=',')
df


/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


,Unnamed: 0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c23,c24,c25,c26,c27,c28,c29,c30,c31,c32
0,267247,NaN,30000.0,30000.0,36 months,19.92,1113.69,Director of Housekeeping,4 years,MORTGAGE,...,7762.630000,4530.50,3232.13,0.0,0.00,0.0000,1113.69,0.0,1.0,D
1,139797,NaN,7000.0,7000.0,36 months,13.56,237.76,Waitress,8 years,RENT,...,940.490000,645.48,295.01,0.0,0.00,0.0000,237.76,0.0,0.0,C
2,519848,NaN,4800.0,4800.0,36 months,16.02,168.81,Owner,10+ years,MORTGAGE,...,2183.850000,1476.07,707.78,0.0,0.00,0.0000,168.81,0.0,0.0,C
3,662736,NaN,35000.0,35000.0,60 months,16.99,869.66,Systems Administrator,1 year,MORTGAGE,...,33916.740000,19296.54,14620.20,0.0,0.00,0.0000,869.66,0.0,0.0,D
4,1239286,37356.0,18000.0,18000.0,36.0,0.1335,609.54,Registered nurse,9 years,MORTGAGE,...,19999.210000,18000.00,1999.21,0.0,0.00,0.0000,13917.16,0.0,0.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063847,787217,NaN,16000.0,16000.0,36 months,9.67,513.80,Staff Accountant,5 years,RENT,...,18493.498361,16000.00,2493.50,0.0,0.00,0.0000,1024.30,0.0,0.0,B
1063848,1141628,127881.0,15000.0,15000.0,36.0,0.1409,513.33,colonial webb contractors,7 years,MORTGAGE,...,11293.260000,8401.48,2881.22,0.0,10.56,0.0000,513.33,0.0,0.0,B
1063849,884569,NaN,19350.0,19350.0,60 months,11.99,430.34,Driver,10+ years,OWN,...,23199.294126,19350.00,3849.29,0.0,0.00,0.0000,14117.04,0.0,1.0,C
1063850,532316,NaN,5000.0,5000.0,36 months,9.93,161.18,Superintendent,1 year,RENT,...,2092.580000,1637.19,455.39,0.0,0.00,0.0000,161.18,0.0,0.0,B


In [2]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063852 entries, 0 to 1063851
Data columns (total 33 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1063852 non-null  int64  
 1   c1          381765 non-null   float64
 2   c2          1063846 non-null  float64
 3   c3          1063846 non-null  float64
 4   c4          1063852 non-null  object 
 5   c5          1063852 non-null  object 
 6   c6          1063846 non-null  float64
 7   c7          1004353 non-null  object 
 8   c8          999740 non-null   object 
 9   c9          1063852 non-null  object 
 10  c10         1063842 non-null  float64
 11  c11         1063852 non-null  object 
 12  c12         1063852 non-null  object 
 13  c13         1063852 non-null  object 
 14  c14         1063852 non-null  object 
 15  c15         1063056 non-null  float64
 16  c16         1063824 non-null  float64
 17  c17         1063824 non-null  float64
 18  c18         1063824 no

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1063852.0,759661.662117,438731.848371,1.000000e+00,379494.500,759527.500,1.139658e+06,1.519789e+06
c1,381765.0,148290.924142,218866.073861,0.000000e+00,34060.000,95419.000,1.634970e+05,1.314167e+06
c2,1063846.0,14895.521015,9059.505592,5.000000e+02,8000.000,12600.000,2.000000e+04,4.000000e+04
c3,1063846.0,14873.462418,9051.447898,5.000000e+02,8000.000,12525.000,2.000000e+04,4.000000e+04
c6,1063846.0,439.953125,260.513405,1.567000e+01,251.060,375.980,5.827100e+02,1.670150e+03
c10,1063842.0,76398.415155,72375.477018,0.000000e+00,45000.000,65000.000,9.029200e+04,9.930475e+06
c15,1063056.0,18.221565,14.150012,0.000000e+00,11.450,17.280,2.369000e+01,3.146033e+03
c16,1063824.0,0.358977,22.936688,0.000000e+00,0.000,0.000,0.000000e+00,1.555328e+04
c17,1063824.0,0.732997,22.941827,0.000000e+00,0.000,0.000,1.000000e+00,1.555328e+04
c18,1063824.0,11.459510,23.528062,0.000000e+00,8.000,10.000,1.400000e+01,1.555328e+04


In [4]:
N = len(df) - 10000
columns = [
    'c1',
    'c2',
    'c3',
    'c6',
    'c13',
    'c15',
    'c18',
    'c21',
    'c23',
    'c24',
    'c25',
    'c32'
]

DataSetTrain, DataSetTest = df.loc[:N, columns].dropna()[:500000], df.loc[N:, columns].dropna()

In [5]:
model = RandomForestClassifier()
test_size = 0.3

model.find_best_params(DataSetTrain, test_size=test_size, n_trials=10)

[I 2022-09-26 01:45:32,501] A new study created in memory with name: no-name-2cf7ee4e-22b2-4d9e-a465-f3479c747140
[I 2022-09-26 02:15:36,397] Trial 0 finished with value: 0.8551271601527272 and parameters: {'n_estimators': 891, 'max_depth': 115, 'max_features': 10}. Best is trial 0 with value: 0.8551271601527272.
[I 2022-09-26 02:32:52,096] Trial 1 finished with value: 0.8587392778093702 and parameters: {'n_estimators': 894, 'max_depth': 72, 'max_features': 6}. Best is trial 1 with value: 0.8587392778093702.
[I 2022-09-26 02:34:23,592] Trial 2 finished with value: 0.8113533049264294 and parameters: {'n_estimators': 173, 'max_depth': 39, 'max_features': 2}. Best is trial 1 with value: 0.8587392778093702.
[I 2022-09-26 02:41:19,385] Trial 3 finished with value: 0.8588691572539474 and parameters: {'n_estimators': 366, 'max_depth': 199, 'max_features': 6}. Best is trial 3 with value: 0.8588691572539474.
[I 2022-09-26 02:45:11,901] Trial 4 finished with value: 0.8160273337071307 and paramet

In [6]:
X, y = DataSetTrain.loc[:, model.features], DataSetTrain.loc[:, model.target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

model.fit(X_train, y_train)
endog = model.predict(X_test) != y_test

In [7]:
MPPDataSetTrain = X_test.copy()
MPPDataSetTrain['c32'] = endog
MPPDataSetTrain

,c1,c2,c3,c6,c15,c18,c21,c23,c24,c25,c32
511730,52092.0,6000.0,6000.0,200.30,9.30,3.0,12.0,5007.500000,3927.27,1080.23,True
632967,35205.0,6000.0,6000.0,207.87,8.49,16.0,31.0,1662.960000,1109.73,549.47,False
211436,16115.0,18000.0,18000.0,578.79,22.43,8.0,43.0,14459.880000,11925.58,2534.30,False
1036691,1216703.0,5000.0,5000.0,152.18,13.26,6.0,22.0,5437.533734,5000.00,437.53,False
57727,191586.0,7375.0,7375.0,256.78,30.48,13.0,36.0,9197.850000,7375.00,1822.85,False
...,...,...,...,...,...,...,...,...,...,...,...
85422,814.0,19425.0,19425.0,468.37,21.42,12.0,22.0,26971.232034,19425.00,7546.23,False
63990,38778.0,12000.0,12000.0,279.10,10.21,7.0,27.0,14586.210001,12000.00,2586.21,False
706406,100086.0,7900.0,7900.0,255.51,12.79,7.0,17.0,8686.050000,7900.00,786.05,False
872770,116672.0,7000.0,7000.0,213.05,7.22,14.0,26.0,7669.749882,7000.00,669.75,False


In [8]:
mpp = XGBClassifier()
test_size = 0.1

mpp.find_best_params(MPPDataSetTrain, test_size=test_size, n_trials=10)

[I 2022-09-26 03:22:54,243] A new study created in memory with name: no-name-59a172ae-921c-458a-9b04-3b2d0cdeb43e
[I 2022-09-26 03:26:09,254] Trial 0 finished with value: 0.8134611740027838 and parameters: {'learning_rate': 0.006717315436128768, 'n_estimators': 489, 'max_depth': 42}. Best is trial 0 with value: 0.8134611740027838.
[I 2022-09-26 03:28:26,780] Trial 1 finished with value: 0.8154170425570766 and parameters: {'learning_rate': 0.004372747386210632, 'n_estimators': 356, 'max_depth': 57}. Best is trial 1 with value: 0.8154170425570766.
[I 2022-09-26 03:35:10,031] Trial 2 finished with value: 0.814924428115725 and parameters: {'learning_rate': 0.005810852005729468, 'n_estimators': 942, 'max_depth': 111}. Best is trial 1 with value: 0.8154170425570766.
[I 2022-09-26 03:37:50,560] Trial 3 finished with value: 0.8138419617422331 and parameters: {'learning_rate': 0.00933806672554455, 'n_estimators': 395, 'max_depth': 175}. Best is trial 1 with value: 0.8154170425570766.
[I 2022-09

In [9]:
X, y = MPPDataSetTrain.loc[:, model.features], MPPDataSetTrain.loc[:, model.target]
mpp.fit(X, y)

In [10]:
X, y = DataSetTest.loc[:, model.features], DataSetTest.loc[:, model.target]

In [11]:
y_true = np.array(model.predict(X) != y, dtype=int)
y_true

array([0, 0, 0, ..., 1, 1, 1])

In [12]:
y_pred = np.array(mpp.predict(X), dtype=int)
sum(y_pred)

108

In [13]:
f1_score(y_true, y_pred, average='weighted')

0.8074172042848797

In [23]:
print('Сколько всего прогнозов: ', len(y_true))
print('Количество истинных меток 0: ', sum(y_true==0))
print('Количество правильных прогнозов с меткой 0: ', sum((y_true == y_pred) & (y_true ==0)))

print('Количество истинных меток 1: ', sum(y_true==1))
print('Количество правильных прогнозов с меткой 1: ', sum((y_true == y_pred) & (y_true ==1)))

#Вывод
#Таким образом было угадано 42 некорректных значений, а ложных срабатываний оказалось не так много

Сколько всего прогнозов:  3515
Количество истинных меток 0:  3015
Количество правильных прогнозов с меткой 0:  2949
Количество истинных меток 1:  500
Количество правильных прогнозов с меткой 1:  42
